In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ===================================================================
# CONFIGURATIONS AND HYPERPARAMETERS (OPTIMIZED FOR TRAINING FROM SCRATCH)
# ===================================================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
# MODIFICATION: Use a single learning rate for the entire model
LEARNING_RATE = 1e-3 
WEIGHT_DECAY = 5e-4
NUM_EPOCHS = 80
EARLY_STOPPING_PATIENCE = 10
LABEL_SMOOTHING = 0.1
GRADIENT_CLIP_VALUE = 1.0
BEST_MODEL_PATH = 'best_lung_cancer_model_resnet18_from_scratch.pth'

# ===================================================================
# DEFINITIONS (Using strong augmentation)
# ===================================================================
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    normalize,
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.2)),
])

val_test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

class LungCancerImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images, self.labels, self.transform = images, labels, transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        image, label = self.images[idx], self.labels[idx]
        if self.transform: image = self.transform(image)
        else: image = transforms.ToTensor()(image)
        label = torch.tensor(label, dtype=torch.long).squeeze()
        return image, label

# ===================================================================
# MAIN EXECUTION BLOCK
# ===================================================================
if __name__ == '__main__':
    # 1. DATA LOADING
    # ===================================================================
    try:
        train = np.load('Dataset5_raw_train.npz')
        val = np.load('Dataset5_raw_val.npz')
        test = np.load('Dataset5_raw_test.npz')
        image_train, label_train = train['image'], train['image_label']
        image_val, label_val = val['image'], val['image_label']
        image_test, label_test = test['image'], test['image_label']
        NUM_CLASSES = len(np.unique(label_train))
        print("Datasets loaded successfully.")
    except FileNotFoundError:
        print("Dataset files not found. Using dummy data for demonstration.")
        NUM_CLASSES = 5
        image_train = np.random.randint(0, 256, size=(200, 224, 224, 3), dtype=np.uint8)
        label_train = np.random.randint(0, NUM_CLASSES, size=(200, 1))
        image_val = np.random.randint(0, 256, size=(40, 224, 224, 3), dtype=np.uint8)
        label_val = np.random.randint(0, NUM_CLASSES, size=(40, 1))
        image_test = np.random.randint(0, 256, size=(40, 224, 224, 3), dtype=np.uint8)
        label_test = np.random.randint(0, NUM_CLASSES, size=(40, 1))

    # 2. CREATE DATALOADERS
    # ===================================================================
    train_dataset = LungCancerImageDataset(image_train, label_train, transform=train_transform)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
    val_dataset = LungCancerImageDataset(image_val, label_val, transform=val_test_transform)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
    test_dataset = LungCancerImageDataset(image_test, label_test, transform=val_test_transform)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # 3. MODEL DEFINITION (ResNet-18 from scratch)
    # ===================================================================
    # Load ResNet-18 with random weights (no pre-training)
    model = models.resnet18(weights=None)
    
    # All layers are trainable by default when training from scratch
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(256, NUM_CLASSES)
    )
    model = model.to(DEVICE)
    print(f"Model is running on device: {DEVICE}")
    print(f"Training ResNet-18 from scratch.")
    print(f"Number of trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

    # 4. LOSS, OPTIMIZER, AND SCHEDULER
    # ===================================================================
    criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)
    
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-7)

    # 5. TRAINING AND VALIDATION LOOP
    # ===================================================================
    best_val_acc = 0.0
    epochs_no_improve = 0

    for epoch in range(NUM_EPOCHS):
        model.train()
        train_running_loss, train_correct, train_total = 0.0, 0, 0
        train_loop = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Training]", leave=False)
        
        for images, labels in train_loop:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP_VALUE)
            optimizer.step()

            train_running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss, train_acc = train_running_loss / train_total, train_correct / train_total

        model.eval()
        val_running_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_dataloader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss, val_acc = val_running_loss / val_total, val_correct / val_total
        
        scheduler.step()

        print(f"Epoch {epoch+1}/{NUM_EPOCHS} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f} | "
              f"LR: {optimizer.param_groups[0]['lr']:.2e}")

        if val_acc >= best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), BEST_MODEL_PATH)
            print(f"  -> New best model saved to {BEST_MODEL_PATH} (Val Acc: {val_acc:.4f})")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= EARLY_STOPPING_PATIENCE:
            print(f"\nEarly stopping triggered after {EARLY_STOPPING_PATIENCE} epochs with no improvement.")
            break

    print("\nTraining finished.")

    # 6. FINAL EVALUATION ON TEST SET
    # ===================================================================
    print("\n--- Running Final Evaluation on Test Set ---")
    try:
        model.load_state_dict(torch.load(BEST_MODEL_PATH))
    except FileNotFoundError:
        print(f"ERROR: Best model file not found at {BEST_MODEL_PATH}. Skipping final evaluation.")
        exit()

    model.eval()

    test_correct, test_total = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_dataloader, desc="        [Standard Test Eval]"):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    test_accuracy_no_tta = test_correct / test_total
    print(f"\nFinal Test Accuracy (NO TTA): {test_accuracy_no_tta:.4f}")

    print("\n--- Running Final Evaluation with Test-Time Augmentation ---")
    tta_transforms = [val_test_transform, transforms.Compose([transforms.ToPILImage(), transforms.Resize(256), transforms.CenterCrop(224), transforms.RandomHorizontalFlip(p=1.0), transforms.ToTensor(), normalize])]
    tta_test_dataset = LungCancerImageDataset(image_test, label_test, transform=None)
    tta_test_dataloader = DataLoader(tta_test_dataset, batch_size=BATCH_SIZE//2, shuffle=False, num_workers=0)
    all_tta_preds, all_labels = [], []
    with torch.no_grad():
        for images_np, labels in tqdm(tta_test_dataloader, desc="        [TTA Eval]"):
            batch_tta_preds = []
            for tta_transform in tta_transforms:
                aug_images = torch.stack([tta_transform(img_np) for img_np in images_np]).to(DEVICE)
                outputs = model(aug_images)
                batch_tta_preds.append(outputs.softmax(dim=1).cpu())
            all_tta_preds.append(torch.stack(batch_tta_preds).mean(dim=0))
            all_labels.append(labels)
    all_tta_preds, all_labels = torch.cat(all_tta_preds), torch.cat(all_labels)
    _, final_preds = torch.max(all_tta_preds, 1)
    tta_accuracy = (final_preds == all_labels).sum().item() / len(all_labels)
    print(f"\nFinal Test Accuracy (with TTA): {tta_accuracy:.4f}")

Datasets loaded successfully.
Model is running on device: cuda
Training ResNet-18 from scratch.
Number of trainable parameters: 11309125


Epoch 1/80 | Train Loss: 1.5107, Train Acc: 0.3836 | Val Loss: 1.4680, Val Acc: 0.3921 | LR: 9.76e-04
  -> New best model saved to best_lung_cancer_model_resnet18_from_scratch.pth (Val Acc: 0.3921)


Epoch 2/80 | Train Loss: 1.4694, Train Acc: 0.4057 | Val Loss: 1.4103, Val Acc: 0.4391 | LR: 9.05e-04
  -> New best model saved to best_lung_cancer_model_resnet18_from_scratch.pth (Val Acc: 0.4391)


Epoch 3/80 | Train Loss: 1.4247, Train Acc: 0.4216 | Val Loss: 1.6561, Val Acc: 0.4010 | LR: 7.94e-04


Epoch 4/80 | Train Loss: 1.3976, Train Acc: 0.4316 | Val Loss: 1.3006, Val Acc: 0.4720 | LR: 6.55e-04
  -> New best model saved to best_lung_cancer_model_resnet18_from_scratch.pth (Val Acc: 0.4720)


...

Epoch 80/80 | Train Loss: 0.7664, Train Acc: 0.8214 | Val Loss: 0.6124, Val Acc: 0.9019 | LR: 1.00e-03

Training finished.

--- Running Final Evaluation on Test Set ---


        [Standard Test Eval]: 100%|██████████| 205/205 [00:33<00:00,  6.07it/s]



Final Test Accuracy (NO TTA): 0.9057

--- Running Final Evaluation with Test-Time Augmentation ---


        [TTA Eval]: 100%|██████████| 409/409 [00:36<00:00, 11.30it/s]


Final Test Accuracy (with TTA): 0.9024
